## 1. 환경 설정

`(1) Env 환경변수`

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [3]:
import re
import os, json
from glob import glob

from textwrap import dedent
from pprint import pprint

import uuid

import warnings
warnings.filterwarnings("ignore")

# 함수설정 

In [4]:
def parse_law(law_text):
    # 서문 분리
    # '^'로 시작하여 '제1장' 또는 '제1조' 직전까지의 모든 텍스트를 탐색 
    preamble_pattern = r'^(.*?)(?=제1장|제1조)'
    preamble = re.search(preamble_pattern, law_text, re.DOTALL)
    if preamble:
        preamble = preamble.group(1).strip()
    
    # 장 분리 
    # '제X장' 형식의 제목과 그 뒤에 오는 모든 조항을 하나의 그룹화 
    chapter_pattern = r'(제\d+장\s+.+?)\n((?:제\d+조(?:의\d+)?(?:\(\w+\))?.*?)(?=제\d+장|부칙|$))'
    chapters = re.findall(chapter_pattern, law_text, re.DOTALL)
    
    # 부칙 분리
    # '부칙'으로 시작하는 모든 텍스트를 탐색 
    appendix_pattern = r'(부칙.*)'
    appendix = re.search(appendix_pattern, law_text, re.DOTALL)
    if appendix:
        appendix = appendix.group(1)
    
    # 파싱 결과를 저장할 딕셔너리 초기화
    parsed_law = {'서문': preamble, '장': {}, '부칙': appendix}
    
    # 각 장 내에서 조 분리
    for chapter_title, chapter_content in chapters:
        # 조 분리 패턴
        # 1. '제X조'로 시작 ('제X조의Y' 형식도 가능)
        # 2. 조 번호 뒤에 반드시 '(항목명)' 형식의 제목이 와야 함 
        # 3. 다음 조가 시작되기 전까지 또는 문서의 끝까지의 모든 내용을 포함
        article_pattern = r'(제\d+조(?:의\d+)?\s*\([^)]+\).*?)(?=제\d+조(?:의\d+)?\s*\([^)]+\)|$)'
        
        # 정규표현식을 이용해 모든 조항을 탐색 
        articles = re.findall(article_pattern, chapter_content, re.DOTALL)
        
        # 각 조항의 앞뒤 공백을 제거하고 결과 딕셔너리에 저장
        parsed_law['장'][chapter_title.strip()] = [article.strip() for article in articles]
    
    return parsed_law

In [5]:
# 파싱 함수를 수정 (장이 없이 조문으로만 구성된 경우)
def parse_law_v2(law_text):
    # 서문 분리
    preamble_pattern = r'^(.*?)(?=제1장|제1조)'
    preamble = re.search(preamble_pattern, law_text, re.DOTALL)
    if preamble:
        preamble = preamble.group(1).strip()
    
    # 장 분리 
    chapter_pattern = r'(제\d+장\s+.+?)\n((?:제\d+조(?:의\d+)?(?:\(\w+\))?.*?)(?=제\d+장|부칙|$))'
    chapters = re.findall(chapter_pattern, law_text, re.DOTALL)
    
    # 부칙 분리
    appendix_pattern = r'(부칙.*)'
    appendix = re.search(appendix_pattern, law_text, re.DOTALL)
    if appendix:
        appendix = appendix.group(1)
    
    parsed_law = {'서문': preamble, '부칙': appendix}
    
    # 조 분리 패턴
    article_pattern = r'(제\d+조(?:의\d+)?\s*\([^)]+\).*?)(?=제\d+조(?:의\d+)?\s*\([^)]+\)|$)'
    
    if chapters:  # 장이 있는 경우
        parsed_law['장'] = {}
        for chapter_title, chapter_content in chapters:
            articles = re.findall(article_pattern, chapter_content, re.DOTALL)
            parsed_law['장'][chapter_title.strip()] = [article.strip() for article in articles]
    else:  # 장이 없는 경우
        # 서문과 부칙을 제외한 본문에서 조문 추출
        main_text = re.sub(preamble_pattern, '', law_text, flags=re.DOTALL)
        main_text = re.sub(appendix_pattern, '', main_text, flags=re.DOTALL)
        articles = re.findall(article_pattern, main_text, re.DOTALL)
        parsed_law['조문'] = [article.strip() for article in articles]
    
    return parsed_law

# 2. 벡터DB에 저장

## 법률문서


In [7]:
os.chdir('/Users/simsewon/Documents/agent_project/NaturalLangProcessingInBigData')

- PDf 문서를 가져와서 조항 별로 구분하여 정리

In [27]:
from langchain_community.document_loaders import PyPDFLoader

pdf_file = '전자상거래 등에서의 소비자보호에 관한 법률(법률)(제20302호)(20250214).pdf'

loader = PyPDFLoader(f'data/{pdf_file}')
pages = loader.load()

len(pages)



18

In [11]:

# 각 페이지의 텍스트를 결합하여 재분리
text_for_delete = r"법제처\s+\d+\s+국가법령정보센터\전자상거래 등에서의 소비자보호에 관한 법률"

law_text = "\n".join([re.sub(text_for_delete, "", p.page_content).strip() for p in pages])

parsed_law = parse_law(law_text)

{'서문': '법제처                                                            1                                                       국가법령정보센터\n전자상거래 등에서의 소비자보호에 관한 법률 \n \n전자상거래 등에서의 소비자보호에 관한 법률 ( 약칭: 전자상거래법 ) \n[시행 2025. 2. 14.] [법률 제20302호, 2024. 2. 13., 일부개정] \n공정거래위원회 (소비자거래정책과) 044-200-4454',
 '장': {'제1장 총칙 <개정 2012. 2. 17.>': ['제1조(목적) 이 법은 전자상거래 및 통신판매 등에 의한 재화 또는 용역의 공정한 거래에 관한 사항을 규정함으로써 소\n비자의 권익을 보호하고 시장의 신뢰도를 높여 국민경제의 건전한 발전에 이바지함을 목적으로 한다.\n[전문개정 2012. 2. 17.]',
   '제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2012. 6. 1.>\n1. “전자상거래”란 전자거래(「전자문서 및 전자거래 기본법」 제2조제5호에 따른 전자거래를 말한다. 이하 같다)의\n방법으로 상행위(商行爲)를 하는 것을 말한다. \n2. “통신판매”란 우편ㆍ전기통신, 그 밖에 총리령으로 정하는 방법으로 재화 또는 용역(일정한 시설을 이용하거나\n용역을 제공받을 수 있는 권리를 포함한다. 이하 같다)의 판매에 관한 정보를 제공하고 소비자의 청약을 받아 재 \n화 또는 용역(이하 “재화등”이라 한다)을 판매하는 것을 말한다. 다만, 「방문판매 등에 관한 법률」 제2조제3호에 \n따른 전화권유판매는 통신판매의 범위에서 제외한다. \n3. “통신판매업자”란 통신판매를 업(業)으로 하는 자 또는 그와의 약정에 따라 통신판매업무를 수행하는 자를 말한\n다. \n4. “통신판매중개”란 사이버몰(컴퓨터 등과 정보통신설비를 이용하여 재화등을 거래할 수 있도록 설정된 가상의 영\n업장을 말한다. 이하 같다)의

In [26]:
parsed_law['부칙']  

'부칙 <제20302호,2024. 2. 13.> \n제1조(시행일) 이 법은 공포 후 1년이 경과한 날부터 시행한다. 다만, 제21조의2제2항 및 제21조의3의 개정규정은 공\n포한 날부터 시행한다.\n제2조(정기결제 대금의 증액 또는 유료 전환 시 소비자 동의 등에 관한 적용례) 제13조제6항의 개정규정은 이 법 시행\n후 재화등의 정기결제 대금이 증액되거나 재화등이 무상으로 공급된 후 유료 정기결제로 전환이 이루어지는 경우\n부터 적용한다.'

In [35]:
# meta정보(source, chapter, name), page_contnet

from langchain_core.documents import Document

law_pdf_docs = []
for law in parsed_law['장'].keys():
    for article in parsed_law['장'][law]:

        # metadata 내용을 정리 
        metadata = {
                "source": pdf_file,
                "chapter": law,
                "name" : "전자상거래소비자보호법"
                }

        # metadata 내용을 본문에 추가 
        content = f"[법률정보]\n다음 조항은 {metadata['name']} {metadata['chapter']}에서 발췌한 내용입니다.\n\n[법률조항]\n{article}"

        law_pdf_docs.append(Document(page_content=content, metadata=metadata))


## 생활법령 QnA
- jsonl파일

In [34]:
qna_json_docs = []
with open("data/customer_law_qa.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        row = json.loads(line)
        
        doc = Document(
            page_content=f"Q: {row['question']}\nA: {row['answer']}",
            metadata={
                "source": row.get("source"),
                "chapter": row.get("category"),
                "name" : "생활법령소비자QnA",
                "url": row.get("url")
                
            }

        )
        qna_json_docs.append(doc)


## 공정거래위원회_소비자민원 

In [36]:
import pandas as pd
from langchain.schema import Document

df = pd.read_csv("data/공정거래위원회_소비자민원.csv", encoding="utf-8")


In [44]:
df['품목명'] =df['품목명'].fillna('')

In [46]:
complain_csv_docs = []
for _, row in df.iterrows():
    doc = Document(
        page_content=f"question: {row['분쟁유형명']}\n answer: {row['answer']}",
        metadata={
            "source": "공정거래위원회_소비자민원",
            "chapter": row.get("품목명", None),
            "name": "생활법령소비자QnA",
         
        }
    )
    complain_csv_docs.append(doc)


In [47]:
complain_csv_docs

[Document(metadata={'source': '공정거래위원회_소비자민원', 'chapter': '', 'name': '생활법령소비자QnA'}, page_content='question: 1) 계량기 고장  계량기 오차과다 등으로 인한 전력량의 과다계량 등 계량 부적정으로 인한 피해\n answer: 차액환급 또는 차액차감정산 합니다.'),
 Document(metadata={'source': '공정거래위원회_소비자민원', 'chapter': '', 'name': '생활법령소비자QnA'}, page_content='question: 1) 계량기 고장  계량기 오차초과 등 계량 부적정으로 인한 피해\n answer: 차액환급 또는 차액차감정산 합니다.'),
 Document(metadata={'source': '공정거래위원회_소비자민원', 'chapter': '도시가스', 'name': '생활법령소비자QnA'}, page_content='question: 1) 계량기 고장  계량기 오차초과 등 계량 부적정으로 인한 피해\n answer: 차액환급 또는 차액차감정산 합니다.'),
 Document(metadata={'source': '공정거래위원회_소비자민원', 'chapter': '', 'name': '생활법령소비자QnA'}, page_content='question: 1) 계약서 미발급\n answer: 계약일로부터 3개월 이내 계약철회(계약금 및 할부금 환급) 입니다. '),
 Document(metadata={'source': '공정거래위원회_소비자민원', 'chapter': '', 'name': '생활법령소비자QnA'}, page_content='question: 1) 공연이 취소되거나 관람일이 연기되어 고객이 입장료의 환급을 요구할 때\n answer: 공연업자의 귀책사유로 취소된 경우에 입장료 환급 및 입장료의 10% 배상이며, 천재지변 등 불가항력의 경우 입장료 환급입니다.(관람권을 할인 판매한 경우에는 거래가격을 기준으로 하되, 이는 사업자

# Chroma 인덱스 생성

In [48]:
final_docs= law_pdf_docs + qna_json_docs + complain_csv_docs

In [49]:
final_docs

[Document(metadata={'source': '전자상거래 등에서의 소비자보호에 관한 법률(법률)(제20302호)(20250214).pdf', 'chapter': '제1장 총칙 <개정 2012. 2. 17.>', 'name': '전자상거래소비자보호법'}, page_content='[법률정보]\n다음 조항은 전자상거래소비자보호법 제1장 총칙 <개정 2012. 2. 17.>에서 발췌한 내용입니다.\n\n[법률조항]\n제1조(목적) 이 법은 전자상거래 및 통신판매 등에 의한 재화 또는 용역의 공정한 거래에 관한 사항을 규정함으로써 소\n비자의 권익을 보호하고 시장의 신뢰도를 높여 국민경제의 건전한 발전에 이바지함을 목적으로 한다.\n[전문개정 2012. 2. 17.]'),
 Document(metadata={'source': '전자상거래 등에서의 소비자보호에 관한 법률(법률)(제20302호)(20250214).pdf', 'chapter': '제1장 총칙 <개정 2012. 2. 17.>', 'name': '전자상거래소비자보호법'}, page_content='[법률정보]\n다음 조항은 전자상거래소비자보호법 제1장 총칙 <개정 2012. 2. 17.>에서 발췌한 내용입니다.\n\n[법률조항]\n제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2012. 6. 1.>\n1. “전자상거래”란 전자거래(「전자문서 및 전자거래 기본법」 제2조제5호에 따른 전자거래를 말한다. 이하 같다)의\n방법으로 상행위(商行爲)를 하는 것을 말한다. \n2. “통신판매”란 우편ㆍ전기통신, 그 밖에 총리령으로 정하는 방법으로 재화 또는 용역(일정한 시설을 이용하거나\n용역을 제공받을 수 있는 권리를 포함한다. 이하 같다)의 판매에 관한 정보를 제공하고 소비자의 청약을 받아 재 \n화 또는 용역(이하 “재화등”이라 한다)을 판매하는 것을 말한다. 다만, 「방문판매 등에 관한 법률」 제2조제3호에 \n따른 전화권유판매는 통신판매의 범위에서 제외한다. \n3. “통신판매업자

In [50]:

from langchain_chroma import Chroma
from langchain_ollama  import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3") 

# Chroma 인덱스 생성
labor_db = Chroma.from_documents(
    documents=final_docs, 
    embedding=embeddings_model,   
    # collection_name="ecommerce_law",
    collection_name="ecommerce_law_v2",
    persist_directory="../chroma_db",
)


